<a href="https://colab.research.google.com/github/dishankkalra23/Medical-Appointment-No-Shows/blob/main/Medical_Appointment_No_Shows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Medical Appointment No Shows

>[Project: Medical Appointment No Shows](#scrollTo=BmfbjWRNOPeO)

>>>[Importing Libraries](#scrollTo=zy3dSfeNPPCc)

>>>[Downloading dataset](#scrollTo=by5KNcWYVbXS)

>>>[Loading dataset](#scrollTo=N14ODJZWXEZd)

>[Data Wrangling](#scrollTo=zzG5_tl6_OJF)

>>[Making new column Scheduled time](#scrollTo=kislWLDHJly8)

>>[TO DO: Handcap and SMS-received issue](#scrollTo=n4nQRBDQb_0q)

>>[Continue](#scrollTo=T9GjygqtcVmp)

>>[Exploratory Data Analysis](#scrollTo=z2dmfUqaOPee)

>>>[Research Question 1 (Replace this header name!)](#scrollTo=z2dmfUqaOPee)

>>>[Research Question 2  (Replace this header name!)](#scrollTo=2HAUXwNDOPef)

>>[Conclusions](#scrollTo=LfvthrGmOPeg)



### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Downloading dataset

In [ ]:
! pip install -q kaggle

In [ ]:
# Upload your kaggle.json file containing API token
from google.colab import files
files.upload()

In [ ]:
# Move the downloaded file to a location ~/.kaggle/kaggle.json
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

# You need to give proper permissions to the file (since this is a hidden folder)
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Copy API command to download dataset
! kaggle datasets download -d joniarroba/noshowappointments
! unzip \*.zip
! rm *.zip

  0% 0.00/2.40M [00:00<?, ?B/s]
100% 2.40M/2.40M [00:00<00:00, 80.1MB/s]
Archive:  noshowappointments.zip
  inflating: KaggleV2-May-2016.csv   


### Loading dataset

In [ ]:
df = pd.read_csv('/content/KaggleV2-May-2016.csv')

In [ ]:
df.sample(5)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
100728,1.857518e+11,5768381,F,2016-06-03T07:32:59Z,2016-06-03T00:00:00Z,33,SÃO CRISTÓVÃO,1,0,0,0,0,0,No
12516,9.398587e+13,5667306,M,2016-05-06T07:38:12Z,2016-05-19T00:00:00Z,21,BELA VISTA,0,0,0,0,0,0,No
51799,8.568853e+12,5740690,F,2016-05-25T15:52:02Z,2016-05-25T00:00:00Z,56,SOLON BORGES,0,0,0,0,0,0,No
100270,3.281118e+13,5784214,F,2016-06-07T15:24:59Z,2016-06-07T00:00:00Z,49,DA PENHA,0,0,0,0,0,0,No
86773,6.431747e+13,5760101,M,2016-06-01T14:18:30Z,2016-06-01T00:00:00Z,62,PARQUE MOSCOSO,0,1,1,0,1,0,No


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


> 

> No missing values in data

> **Data type of columns to be fixed,**
1. PatientId is the unique id for identification of a patient and float datatype doesn't make sense.
2. ScheduledDay and AppointmentDay can be used in analysis if we convert them into date-time format.


# Data Wrangling

In [ ]:
df.PatientId = df.PatientId.astype('int')
df.PatientId.dtypes

dtype('int64')

In [ ]:
df.ScheduledDay = pd.to_datetime(df.ScheduledDay)
df.AppointmentDay = pd.to_datetime(df.AppointmentDay)
df[['ScheduledDay','AppointmentDay']].dtypes

ScheduledDay      datetime64[ns, UTC]
AppointmentDay    datetime64[ns, UTC]
dtype: object

## Making new column Scheduled time 
> Converting ScheduleDate & AppointmentDate column to store dates and ScheduledTime & AppointmentTime to  store time of appointment

In [ ]:
df['ScheduledTime'] = pd.to_datetime(df.ScheduledDay).dt.time
df['AppointmentTime'] = pd.to_datetime(df.AppointmentDay).dt.time

In [ ]:
df['ScheduledDay'] = df['ScheduledDay'].dt.date
df['AppointmentDay'] = df['AppointmentDay'].dt.date

In [ ]:
df.AppointmentTime.sample(5)

62249    00:00:00
4102     00:00:00
70328    00:00:00
12321    00:00:00
50472    00:00:00
Name: AppointmentTime, dtype: object

In [ ]:
df.AppointmentTime.nunique()

1

> Appointment Time is 00:00:00 in all the rows, it doesn't provide any relevance to analysis. Hence dropping AppointmentTime column 

In [ ]:
df.drop(columns='AppointmentTime',inplace=True)
df.columns

Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show',
       'ScheduledTime'],
      dtype='object')

In [ ]:
df.Age.describe()

count    110527.000000
mean         37.088874
std          23.110205
min          -1.000000
25%          18.000000
50%          37.000000
75%          55.000000
max         115.000000
Name: Age, dtype: float64

> Age can never be negative, removing rows which have age less than 0

In [ ]:
df_less_0 = df.query('Age < 0')
df_less_0.Age.count()

1

> Only single row with age less than 0

In [ ]:
df = df.query('Age >= 0')

## TO DO: Handcap and SMS-received issue

In [ ]:
df.Handcap.unique()

array([0, 1, 2, 3, 4])

In [ ]:
df.SMS_received.unique()

array([0, 1])

In [ ]:
x = df.query('SMS_received >= 1').SMS_received
x.count()/df.SMS_received.count()



0.3210285362720084

## Continue

In [218]:
df['No-show'].unique()

array(['No', 'Yes'], dtype=object)

In [ ]:
df.columns

Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show',
       'ScheduledTime'],
      dtype='object')

## Rename columns

In [ ]:
labels = {'PatientId':"patient_id", 'AppointmentID':'appointment_id', 'Gender':'gender', 
        'ScheduledDay':'scheduled_day',
       'AppointmentDay':'appointment_day', 'Age':'age', 'Neighbourhood':'neighbourhood', 
       'Scholarship':'scholarship', 'Hipertension':'hypertension',
       'Diabetes':'diabetes', 'Alcoholism':'alcoholism', 'Handcap':'handicap', 
       'SMS_received':'sms_received', 'No-show':'show',
       'ScheduledTime':'scheduled_time'}
df.rename(columns=labels,inplace=True)

In [232]:
df.columns

Index(['patient_id', 'appointment_id', 'gender', 'scheduled_day',
       'appointment_day', 'age', 'neighbourhood', 'scholarship',
       'hypertension', 'diabetes', 'alcoholism', 'handicap', 'sms_received',
       'show', 'scheduled_time'],
      dtype='object')

## Changing no_show(to show) column to avoid misconception and easily understandable

Substituting Yes with 0 and No with 1

In [234]:
df.show = df.show.map({'Yes':0,'No':1})
df.show.astype(int)

## Duplicates in data

In [239]:
df.duplicated().sum()

0

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!